### Functions for stepFunction File

In [ ]:
# Create Functions

def createList(r1, r2):
    # Create a list of the months betweeen planting and maturity
    return np.arange(r1, r2+1)

def getListcom(p, m):
    # check if planting and maturity are in the same year
    if np.any(p < m):
        return createList(p, m)
    else:
        return np.concatenate((createList(p, 12), createList(1, m)))

def rank_crops(df_area):
#     print(df_area)
    df_area = df_area.sort_values('Growing_area', ascending=False)
    df_area['rank'] = range(1, len(df_area) + 1)
#     print(df_area)
    return df_area.set_index('rank')

def gdfmask(gdf, raster_path, all_touched=True):
  """ returns a raster mask of the gdf
  
  Parameters
    ----------
    gdf : geopandasdataframe object
        Country
    raster : path to tiff raster
        Administration level

    Returns
    -------
    dictionary of geopandas dataframe's rasters with characteristics of given raster
  
  
  """
  
  with rasterio.open(raster_path) as src:
    # Read the raster data into a NumPy array
    raster_shape = src.read(1).shape
    raster_crs = src.crs
    raster_transform = src.transform

  gdf0 = gdf.to_crs(src.crs)
  gdf_masks = {}
  for index, row in gdf0.iterrows():
    # print(shapely_geometry)
    gdf_masks[index] = geometry_mask([mapping(row['geometry'])],
                                     out_shape=raster_shape,
                                     transform=raster_transform,
                                     all_touched=all_touched, 
                                     invert=True)
  return(gdf_masks)

In [ ]:
# Create Fuctions for parts of the step function

# Functions for parts of MIRCA steps

def calculate_AD_area(dist_array, ranked_df_ir_rank):
  """
  Calculate AD_Area
  
  """
  # CALCULATE AD
  AD_area = np.where(dist_array["dist_array_ara"] > 0,
                      (AH_array * dist_array["dist_array_aei"]) / dist_array["dist_array_ara"], 0)
    
  # ADJUST AD IF MORE THAN ONE CROP
  if ranked_df_ir_rank.Subcrop.tolist()[0] > 1: 
    AD_area = AD_area/ranked_df_ir_rank.Subcrop.tolist()[0] 

  return(AD_area)


def adjust_AD_area(dist_array, df_tot, AH_array, AD_area, ATS_2, listcom, r_s, c_s):
  """
  Adjust AD_Area
  
  """
  # Filter the total dataframe for a month in which crop grow
  df_max_filt = df_tot[df_tot["Month"].isin(listcom)]
      
  # Group the cell values
  n = len(df_max_filt) // (r_s * c_s)

  # Select the maximum total area assigned for the month in which this crop grow
  df_max = df_max_filt.groupby(np.arange(len(df_max_filt)) // n).max()
    
  # Change it to array
  max_step2 = np.array(df_max["Total_st1"]).reshape(r_s, c_s)

  # Determine the free AEI cell
  AEI_fr_ce_2 = np.where(dist_array["dist_array_aei"] > max_step2, 
                           (dist_array["dist_array_aei"]-max_step2), 0)

  if step in [2]:
  # Determine the AH_free_cell
    AH_fr_ce_2 = np.maximum(AH_array - AD_area, 0)
      
  # Potential area to be distributed in this step
    Pot_ATS2 = np.minimum(AEI_fr_ce_2, AH_fr_ce_2)
        
  if step in [3]:
    Pot_ATS2 = np.where(dist_array["dist_array_cle"]>0, AEI_fr_ce_2, 0)

  if step in [4]:
    Pot_ATS2 = np.where(dist_array["dist_array_cle"] == 0, AEI_fr_ce_2, 0)


  # Determine the distribution ratio as the ratio of ATS and Pot_ATS
  with np.errstate(divide='ignore'):
    AD_ra2 = np.where(ATS_2<np.sum(Pot_ATS2),(ATS_2/np.sum(Pot_ATS2)), 1)
    
  # Exact area to be distributed at this step
  if ATS_2>0:
    AD_area2=Pot_ATS2*AD_ra2
    #Area to be distributed in step 1 and 2
                
  else:
    AD_area2=Pot_ATS2*0
      #Area to be distributed in step 1 and 2
      #                 TAD_12=AD_area1

  # TAD_12=AD_area + AD_area2
  AD_area=AD_area + AD_area2

  return(AD_area)


In [ ]:
# Create the step function


def step00(step, ranked_df_ir_rank, ranked_df_rf_rank, 
           dist_array, AH_array, AD_area,
           c_s=None, r_s=None,
           df_mon=None, df_tot = None, TAD_12=None, df_cumulative=None, 
           ATS_2=None, verbose=False,
           template_tiff= None):
  """

  MIRCA Steps Function

  Function to perform any one of the MIRCA steps
  """

  # Get the growing months 
  p = ranked_df_ir_rank.Planting_Month.tolist()[0]
  m = ranked_df_ir_rank.Maturity_Month.tolist()[0]
  listcom = getListcom(p, m)


  # If step 1 calculate AD_area
  if step in [1]:
    # CALCULATE AD
    AD_area = calculate_AD_area(dist_array, ranked_df_ir_rank)



  # If step 2, 3, or 4 Adjust AD_area
  if step in [2, 3, 4]:
    AD_area = adjust_AD_area(dist_array, df_tot, AH_array, AD_area, ATS_2, listcom, r_s, c_s)



  if step in [1, 2]: #, 2
    if round(np.sum(AD_area),2) <= (ranked_df_ir_rank.Growing_area.tolist()[0]):
    # if round(np.sum(TAD_12),2) < (ranked_df_ir_rank.Growing_area.tolist()[0]):
      ATS_2= (ranked_df_ir_rank.Growing_area.tolist()[0])-round(np.sum(AD_area),2)
      # ATS_3= (ranked_df_ir_rank.Growing_area.tolist()[0])-round(np.sum(TAD_12),2)
      AD_area=AD_area
      # AD_area2=TAD_12
      if verbose:
        print("Area distributed after step {}:".format(step), round(np.sum(AD_area),2))
        print("Iteration continues")
      if round(np.sum(AD_area),2)==(ranked_df_ir_rank.Growing_area.tolist()[0]):
        if verbose:
          print("Area to be distributed in next step (step {}):".format(step+1), ATS_2)
          print("Iteration completed ")
    else:
      ATS_2= 0
      # ATS_3= 0
      if ranked_df_ir_rank.Growing_area.tolist()[0] >0:
        if verbose:
          print("Error check your code!")
          print("More area available than the growing area")
        ratio= ranked_df_ir_rank.Growing_area.tolist()[0]/np.sum(AD_area)
    #                             print(ratio)
        AD_area=AD_area*ratio
        # AD_area2=TAD_12*ratio
          
        if verbose:
          print("Iteration completed")
          print("Area distributed in step {}:".format(step),round( np.sum(AD_area),2))
      if ranked_df_ir_rank.Growing_area.tolist()[0]== 0:
        AD_area=AD_area*0
        # AD_area2=TAD_12*0
        if verbose:
          print("No area will be distributed in step {} (Zero harvested area)".format(step))



  if step in [1, 2, 3, 4]: 
    # Call the array (area) to be distributed monthly
    arr1 = AD_area
    arr1_re = np.repeat(arr1, 12)
    c_s = len(arr1[0])
    r_s = len(arr1)
  
    
  # Call the array to be distributed in step1 and convert it to dataframe
  array_name = "Array_" + ranked_df.Crop.tolist()[0][:3] + ranked_df.Crop.tolist()[0][-1] + "1"
    
  # Create a new dataframe in which we will iterate each cell value for each month according to the crop calendar
  #             crop_sum = 0
  df_month = pd.DataFrame(np.arange(1, 13), columns=['Month'])
  df_mon = pd.concat([df_month] * c_s * r_s, ignore_index=True)
    
  # Change the array to dataframe
  df_1 = pd.DataFrame(arr1_re, index=np.arange(len(df_mon)), columns=[array_name])
  # Merge the month and array dataframe
  df_area1 = pd.concat([df_mon, df_1], axis=1)
  # Filter the month in which the crop grows according to the crop calendar
  crop_1 = df_area1[(df_area1["Month"].isin(listcom))]
  # Filter the month in which the crop didn't grow according to the crop calendar
  crop_not = df_area1[(~df_area1["Month"].isin(listcom))]
  # Set the value of the array to 0 for months when the crop doesn't grow
  crop_not[array_name].values[:] = 0
  # Merge the two dataframes and sort it based on index
  df_name = ranked_df.Crop.tolist()[0][:3] + ranked_df.Crop.tolist()[0][-1] + "_step1"
  df_name = pd.concat([crop_1, crop_not])
  df_name = df_name.sort_index()
#             df_name = np.minimum(df_name, dist_array_aei)

    
  # Assign the area to be distributed in this step for each month according to crop calendar
  df_list = []
  df_list.append(df_name[array_name])

  Step_1 = pd.concat(df_list, axis=1).sum(axis=1)

  if df_cumulative is None:
    df_cumulative = pd.Series(np.zeros(len(Step_1)), index=Step_1.index)
#             if 'df_cumulative_st1' not in locals() or df_cumulative_st1.empty:
#                 df_cumulative_st1 = Step_1.copy()  
#             else:
    
  df_cumulative += Step_1
  if step in [4]:
    df_irrigaed=pd.concat([df_mon, Step_1], axis=1)
    df_irrigaed.columns = ['Month', 'Total_st1']
    
  df_tot = pd.concat([df_mon, df_cumulative], axis=1)
  df_tot.columns = ['Month', 'Total_st1']
    
      

  if step in [4]:
    if round(np.sum(AD_area)) < (ranked_df_ir_rank.Growing_area.tolist()[0]):
      print("Error Check Your Code! (full area not assigned)")
      print("Area distributed after step 4:", round(np.sum(AD_area)))
    elif round(np.sum(AD_area))==(ranked_df_ir_rank.Growing_area.tolist()[0]):
      print("Iteration completed ")
      print("Area distributed after step 4:", round(np.sum(AD_area)))
    else:
      # RAINFED CHECK FIX
      if (round(np.sum(AD_area)) - (ranked_df_rf.Growing_area.tolist()[0]))<1:
        print("Iteration completed ")
        print("Area distributed after step4", round(np.sum(AD_area)))
      if ranked_df_ir_rank.Growing_area.tolist()[0] > 0:
        output_folder = "../scratch"

        # Open the existing file to get geotransform and projection
        template_tiff_dataset = gdal.Open(template_tiff)

        # Get geotransform and projection from the input dataset
        geotransform = template_tiff_dataset.GetGeoTransform()
        projection = template_tiff_dataset.GetProjection()
          
        # # Set up the output file
        print(listcom)
        for k in listcom:
          df_max_filt4 = df_irrigaed[(df_irrigaed["Month"] == int(k))]
          arr_filenam = np.array(df_max_filt4["Total_st1"]).reshape(r_s, c_s)
          mask = np.logical_or(np.isnan(arr_filenam), np.logical_or(np.isinf(arr_filenam), arr_filenam == np.NINF))
          arr_filenam = np.where(mask, 0, arr_filenam)
          
            
          output_tiff_path = "{0}/{1}/District/{2}/{3}/{4}/{5}.tiff".format(output_folder, 
                                                   "Irrigated",
                                                   year,
                                                   unit_code,
                                                   crop_name,
                                                   calendar.month_name[k]               
                                                   )
          print(output_tiff_path)
         
          # Create the output directory if it doesn't exist
          output_directory = os.path.dirname(output_tiff_path)
          if not os.path.exists(output_directory):
            os.makedirs(output_directory)

            
          driver = gdal.GetDriverByName("GTiff")
          out_tiff = driver.Create(output_tiff_path, arr_filenam.shape[1], 
                                   arr_filenam.shape[0], 1, gdal.GDT_Byte)
          print(out_tiff)
          # Set the geotransform and projection
          out_tiff.SetGeoTransform(geotransform)
          out_tiff.SetProjection(projection)

          # Write the array to the TIFF file
          out_band = out_tiff.GetRasterBand(1)
          out_band.WriteArray(arr_filenam)

          # Flush data to disk and close the file
          out_band.FlushCache()
          out_tiff = None
          print("TIFF file saved successfully @ {}".format(output_tiff_path))

    
  
  return {"df_cumulative":df_cumulative, "AD_area" : AD_area, 
          'df_mon':df_mon, "ATS_2": ATS_2, "df_tot":df_tot,
          "c_s":c_s, "r_s":r_s}